In [2]:
!pip install xgboost

  Using cached xgboost-2.1.4-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-2.1.4-py3-none-manylinux2014_x86_64.whl (4.5 MB)


In [9]:
import boto3
 
s3 = boto3.client("s3")
bucket_name = "fr-detector"
prefix = "balanced_train/"
 
# List files in the 'balanced_train/' folder
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
 
# Print all file names in the folder
if "Contents" in response:
    print("Files in 'balanced_train/' folder:")
    for obj in response["Contents"]:
        print(obj["Key"])
else:
    print("No files found in 'balanced_train/' folder.")

[03/18/25 13:32:30] INFO     Found credentials from IAM Role:                                   ]8;id=335852;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=86892;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Files in 'balanced_train/' folder:
balanced_train/X_train.csv
balanced_train/processed_train.csv
balanced_train/train.csv
balanced_train/y_train.csv


In [10]:
import pandas as pd
 
# Load CSV from S3 into DataFrame
file_path = f"s3://fr-detector/balanced_train/processed_train.csv"
df = pd.read_csv(file_path)
 
# Display basic information
print(f"Dataset Shape: {df.shape}")  # (rows, columns)
print("\nColumn Names:\n", df.columns.tolist())

Dataset Shape: (341176, 31)

Column Names:
 ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']


In [11]:
import boto3
 
sm_client = boto3.client("sagemaker")
 
feature_group_name = "credit_card_fraud_features"  # Your feature group name
 
response = sm_client.describe_feature_group(FeatureGroupName=feature_group_name)
feature_definitions = response["FeatureDefinitions"]
 
# Extract column names
stored_features = [feature["FeatureName"] for feature in feature_definitions]
 
print(f"📝 Features in Feature Store:\n{stored_features}")
print(f"📊 Total Features in Feature Store: {len(stored_features)}")

📝 Features in Feature Store:
['V14', 'V4', 'V11', 'V9', 'V7', 'V2', 'V18', 'V1', 'V6', 'V5', 'V19', 'Class', 'record_id', 'event_time']
📊 Total Features in Feature Store: 14


In [12]:
df["record_id"]=df.index

In [13]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'record_id'],
      dtype='object')

In [14]:
import pandas as pd
import boto3
 
# Initialize SageMaker Feature Store Runtime client
featurestore_runtime = boto3.client("sagemaker-featurestore-runtime")
 
# Define Feature Group name
feature_group_name = "credit_card_fraud_features"  # Update if needed
 
# List of Record IDs (should be from your dataset)
record_ids = ["1", "2", "3"]  # Replace with actual IDs
 
# Fetch records from Feature Store
records = []
for record_id in record_ids:
    response = featurestore_runtime.get_record(
        FeatureGroupName=feature_group_name,
        RecordIdentifierValueAsString=record_id
    )
    
    if "Record" in response:
        record_dict = {item["FeatureName"]: item["ValueAsString"] for item in response["Record"]}
        records.append(record_dict)
 
# Convert to DataFrame
df_feature_store = pd.DataFrame(records)
 
# Display data
print(df_feature_store.head())

                  V14                  V4                V11  \
0  -0.253266461227142   0.330155451931532  0.624995774387124   
1   -1.19334711605856   -2.01777181381093  -1.94507005977289   
2  -0.640248865304589  -0.709896721404095  0.521930561808433   

                   V9                  V7                  V2  \
0    1.05456029821935  -0.627977906310739  -0.380782710937387   
1  0.0403783170625422   0.017959435638393  -0.626942768707407   
2  -0.131275441288599   0.787857599353298   0.820565649655871   

                 V18                  V1                   V6  \
0  0.651665792250448    1.95504092199146  -0.0861974531704895   
1   2.10686601839936  -0.400975238728654     0.16831047105142   
2  0.521918858041672  0.0725090163689562   -0.359429496231352   

                   V5                V19 Class record_id            event_time  
0  -0.509374248165253  0.167986640374626     0         1  2025-03-07T07:20:34Z  
1  -0.107768780879015  -1.69278043647559     0         2  2

In [15]:
############## testing index##############

import boto3
import pandas as pd
 
# Initialize SageMaker Feature Store Runtime client
featurestore_runtime = boto3.client("sagemaker-featurestore-runtime")
 
# Feature Group name
feature_group_name = "credit_card_fraud_features"
 
# Initialize empty list to store records
records = []
 
# 1️⃣ Query Feature Store to get ALL `record_id` values
# Fetch all records in batches (assuming record_id is a unique key)
response = featurestore_runtime.batch_get_record(Identifiers=[
    {"FeatureGroupName": feature_group_name, "RecordIdentifiersValueAsString": ["1", "2", "3"]}  # Placeholder, we need to get all IDs dynamically
])
 
# 2️⃣ Extract record_id values
for record in response.get("Records", []):
    record_dict = {item["FeatureName"]: item["ValueAsString"] for item in record["Record"]}
    records.append(record_dict)
 
# 3️⃣ Convert to DataFrame
df_feature_store = pd.DataFrame(records)
 
# Display feature store data
print("Feature Store Data Sample:")
print(df_feature_store.head())
 

Feature Store Data Sample:
                  V14                  V4                V11  \
0   -1.19334711605856   -2.01777181381093  -1.94507005977289   
1  -0.640248865304589  -0.709896721404095  0.521930561808433   
2  -0.253266461227142   0.330155451931532  0.624995774387124   

                   V9                  V7                  V2  \
0  0.0403783170625422   0.017959435638393  -0.626942768707407   
1  -0.131275441288599   0.787857599353298   0.820565649655871   
2    1.05456029821935  -0.627977906310739  -0.380782710937387   

                 V18                  V1                   V6  \
0   2.10686601839936  -0.400975238728654     0.16831047105142   
1  0.521918858041672  0.0725090163689562   -0.359429496231352   
2  0.651665792250448    1.95504092199146  -0.0861974531704895   

                   V5                V19 Class record_id            event_time  
0  -0.107768780879015  -1.69278043647559     0         2  2025-03-07T07:20:34Z  
1    1.08039935724521  0.1987722

In [16]:
print(df.dtypes)
print(df_feature_store.dtypes)

Time         float64
V1           float64
V2           float64
V3           float64
V4           float64
V5           float64
V6           float64
V7           float64
V8           float64
V9           float64
V10          float64
V11          float64
V12          float64
V13          float64
V14          float64
V15          float64
V16          float64
V17          float64
V18          float64
V19          float64
V20          float64
V21          float64
V22          float64
V23          float64
V24          float64
V25          float64
V26          float64
V27          float64
V28          float64
Amount       float64
Class          int64
record_id      int64
dtype: object
V14           object
V4            object
V11           object
V9            object
V7            object
V2            object
V18           object
V1            object
V6            object
V5            object
V19           object
Class         object
record_id     object
event_time    object
dtype: object


In [17]:
df_feature_store["record_id"]=df_feature_store["record_id"].astype(int)
df_merged=df.merge(df_feature_store,on="record_id",how="left")

In [18]:
df_merged.head()

,Time,V1_x,V2_x,V3,V4_x,V5_x,V6_x,V7_x,V8,V9_x,...,V9_y,V7_y,V2_y,V18_y,V1_y,V6_y,V5_y,V19_y,Class_y,event_time
0,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,1.05456029821935,-0.627977906310739,-0.380782710937387,0.651665792250448,1.95504092199146,-0.0861974531704895,-0.509374248165253,0.167986640374626,0,2025-03-07T07:20:34Z
2,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,0.0403783170625422,0.017959435638393,-0.626942768707407,2.10686601839936,-0.400975238728654,0.16831047105142,-0.107768780879015,-1.69278043647559,0,2025-03-07T07:20:34Z
3,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,-0.131275441288599,0.787857599353298,0.820565649655871,0.521918858041672,0.0725090163689562,-0.359429496231352,1.08039935724521,0.198772253039242,0,2025-03-07T07:20:34Z
4,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_merged.columns

Index(['Time', 'V1_x', 'V2_x', 'V3', 'V4_x', 'V5_x', 'V6_x', 'V7_x', 'V8',
       'V9_x', 'V10', 'V11_x', 'V12', 'V13', 'V14_x', 'V15', 'V16', 'V17',
       'V18_x', 'V19_x', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26',
       'V27', 'V28', 'Amount', 'Class_x', 'record_id', 'V14_y', 'V4_y',
       'V11_y', 'V9_y', 'V7_y', 'V2_y', 'V18_y', 'V1_y', 'V6_y', 'V5_y',
       'V19_y', 'Class_y', 'event_time'],
      dtype='object')

In [20]:
df_merged = df_merged.drop(columns=[col for col in df_merged.columns if col.endswith('_y')])
df_merged.columns = df_merged.columns.str.replace('_x', '')  # Rename `_x` columns

In [21]:
print(df_merged.columns)
print(df_merged.dtypes)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'record_id', 'event_time'],
      dtype='object')
Time          float64
V1            float64
V2            float64
V3            float64
V4            float64
V5            float64
V6            float64
V7            float64
V8            float64
V9            float64
V10           float64
V11           float64
V12           float64
V13           float64
V14           float64
V15           float64
V16           float64
V17           float64
V18           float64
V19           float64
V20           float64
V21           float64
V22           float64
V23           float64
V24           float64
V25           float64
V26           float64
V27           float64
V28           float64
Amount        float64
Class           int64
record_id       int64
even

In [22]:
df_merged.isnull().sum()

Time               0
V1                 0
V2                 0
V3                 0
V4                 0
V5                 0
V6                 0
V7                 0
V8                 0
V9                 0
V10                0
V11                0
V12                0
V13                0
V14                0
V15                0
V16                0
V17                0
V18                0
V19                0
V20                0
V21                0
V22                0
V23                0
V24                0
V25                0
V26                0
V27                0
V28                0
Amount             0
Class              0
record_id          0
event_time    341173
dtype: int64

In [23]:
df_merged['Class'].value_counts()

0    227451
1    113725
Name: Class, dtype: int64

In [24]:
print(df_merged.head())

       Time        V1        V2        V3        V4        V5        V6  \
0  143352.0  1.955041 -0.380783 -0.315013  0.330155 -0.509374 -0.086197   
1  117173.0 -0.400975 -0.626943  1.555339 -2.017772 -0.107769  0.168310   
2  149565.0  0.072509  0.820566 -0.561351 -0.709897  1.080399 -0.359429   
3   93670.0 -0.535045  1.014587  1.750679  2.769390  0.500089  1.002270   
4   82655.0 -4.026938  1.897371 -0.429786 -0.029571 -0.855751 -0.480406   

         V7        V8        V9  ...       V23       V24       V25       V26  \
0 -0.627978  0.035994  1.054560  ...  0.053208 -0.278602 -0.044999 -0.216780   
1  0.017959 -0.401619  0.040378  ...  0.113442 -1.004095 -1.176695  0.361924   
2  0.787858  0.117276 -0.131275  ...  0.083391  0.148178 -0.431459  0.119690   
3  0.847902 -0.081323  0.371579  ... -0.072754  0.448192 -0.655203 -0.181038   
4 -0.435632  1.313760  0.536044  ...  0.250717  0.066399  0.470787  0.245335   

        V27       V28    Amount  Class  record_id            event_t

In [25]:
df_merged.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'record_id', 'event_time'],
      dtype='object')

In [26]:
df_merged.to_csv("train_final.csv",index=False,header=True)

In [27]:
import pandas as pd
df_test=pd.read_csv("train_final.csv")
print(df_test.head())

       Time        V1        V2        V3        V4        V5        V6  \
0  143352.0  1.955041 -0.380783 -0.315013  0.330155 -0.509374 -0.086197   
1  117173.0 -0.400975 -0.626943  1.555339 -2.017772 -0.107769  0.168310   
2  149565.0  0.072509  0.820566 -0.561351 -0.709897  1.080399 -0.359429   
3   93670.0 -0.535045  1.014587  1.750679  2.769390  0.500089  1.002270   
4   82655.0 -4.026938  1.897371 -0.429786 -0.029571 -0.855751 -0.480406   

         V7        V8        V9  ...       V23       V24       V25       V26  \
0 -0.627978  0.035994  1.054560  ...  0.053208 -0.278602 -0.044999 -0.216780   
1  0.017959 -0.401619  0.040378  ...  0.113442 -1.004095 -1.176695  0.361924   
2  0.787858  0.117276 -0.131275  ...  0.083391  0.148178 -0.431459  0.119690   
3  0.847902 -0.081323  0.371579  ... -0.072754  0.448192 -0.655203 -0.181038   
4 -0.435632  1.313760  0.536044  ...  0.250717  0.066399  0.470787  0.245335   

        V27       V28    Amount  Class  record_id            event_t

/tmp/ipykernel_22029/391477044.py:2: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test=pd.read_csv("train_final.csv")


In [29]:
df_test.shape

(341176, 33)

In [30]:
df_test["event_time"].value_counts()

2025-03-07T07:20:34Z    3
Name: event_time, dtype: int64

In [31]:
df_test.drop(columns=["event_time"],inplace=True)

In [32]:
df_test.columns


Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'record_id'],
      dtype='object')

In [33]:
df_test.to_csv("train_final_1.csv",index=False, header=True)

In [36]:
import boto3
s3_client=boto3.client("s3")
s3_client.upload_file("train_final_1.csv","fr-detector","balanced_train/train_final_1.csv")

In [35]:
!pip install sagemaker boto3 numpy pandas --quiet
 
import sagemaker
import boto3
import numpy as np
import pandas as pd
import os
from sagemaker import get_execution_role
from datetime import datetime

In [36]:
# Define S3 bucket and dataset paths
bucket = "fr-detector"
prefix = "balanced_train"  # Folder where dataset is stored
train_uri = f"s3://{bucket}/{prefix}/processed_train.csv"
 
# SageMaker setup
role = get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()

In [37]:
image_uri = sagemaker.image_uris.retrieve(region=region, framework="xgboost", version="latest")
print("Using XGBoost Image:", image_uri)

[03/12/25 12:33:29] INFO     Ignoring unnecessary instance type: None.                            ]8;id=117867;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=514835;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

Using XGBoost Image: 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [83]:
df_merged.to_csv("train_final.csv", index=False, header=False)

In [84]:
import boto3
 
s3_client = boto3.client("s3")
s3_client.upload_file("train_final.csv", "fr-detector", "balanced_train/train_final.csv")
 
print("Updated training file uploaded to S3.")

Updated training file uploaded to S3.


In [40]:
from sagemaker.experiments.experiment import Experiment
from sagemaker.session import Session
 
experiment_name = "fraud-detection-experiment"
sagemaker_session = Session()
 
# Create experiment (if it doesn't exist)
try:
    experiment = Experiment.load(experiment_name)
    print(f"Loaded existing experiment: {experiment_name}")
except Exception as e:
    if "ResourceNotFound" in str(e):  # Experiment doesn't exist, create it
        experiment = Experiment.create(
            experiment_name=experiment_name,
            description="Credit Card Fraud Detection Experiment",
        )
        print(f"Created new experiment: {experiment_name}")
    else:
        raise  # Reraise any other error

Created new experiment: fraud-detection-experiment


In [76]:
import sagemaker
from sagemaker.experiments.run import Run
from sagemaker.experiments.experiment import Experiment
from sagemaker.experiments.trial_component import _TrialComponent  # Use _TrialComponent
from sagemaker.session import Session

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Define Experiment Name
experiment_name = "fraud-detection-experiment"

# Create the experiment (if it doesn't exist)
try:
    experiment = Experiment.load(experiment_name=experiment_name, sagemaker_session=sagemaker_session)
    print(f"Loaded existing experiment: {experiment_name}")
except:
    experiment = Experiment.create(experiment_name=experiment_name, sagemaker_session=sagemaker_session)
    print(f"Created new experiment: {experiment_name}")

# Create a trial component for this run
trial_component_name = f"trial-component-{sagemaker.utils.unique_name_from_base('xgboost')}"
trial_component = _TrialComponent.create(
    trial_component_name=trial_component_name,
    display_name="XGBoostTrialComponent",  # Modified display name to meet constraints
    sagemaker_session=sagemaker_session
)
print(f"Created trial component: {trial_component_name}")

Loaded existing experiment: fraud-detection-experiment
Created trial component: trial-component-xgboost-1741786740-21e1


In [77]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
import boto3
 
# Get XGBoost image URI
region = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")
 
# Define the estimator
xgb_estimator = Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    output_path="s3://fr-detector/models/",
    sagemaker_session=sagemaker_session
)
 
# Set hyperparameters
xgb_estimator.set_hyperparameters(
    objective="binary:logistic",  # Since it's fraud detection (binary classification)
    num_round=100,                # Number of boosting rounds
    eval_metric="auc",
    verbosity=2
)
 
# Define S3 training input
train_data = TrainingInput(
    "s3://fr-detector/balanced_train/train_final.csv",
    content_type="text/csv"
)

[03/12/25 13:40:02] INFO     Ignoring unnecessary instance type: None.                            ]8;id=704843;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=539610;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [81]:
!aws s3 ls s3://fr-detector/balanced_train/

2025-03-07 05:08:26  185655083 X_train.csv
2025-03-12 11:23:46  190159900 processed_train.csv
2025-03-07 11:15:54  185400270 train.csv
2025-03-12 12:49:30  191675855 train_final.csv
2025-03-07 05:08:29     682358 y_train.csv


In [82]:
!aws s3 cp s3://fr-detector/balanced_train/train_final.csv

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: paths


In [72]:
import sagemaker.experiments.trial_component as trial_component_module
print(dir(trial_component_module))

['ClientError', 'TrialComponentSearchResult', '_TrialComponent', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_api_types', '_base_types', 'absolute_import', 'format_tags', 'time']


In [68]:
import sagemaker.experiments.trial as trial_module
print(dir(trial_module))

['ClientError', '_Trial', '_TrialComponent', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_api_types', '_base_types', 'absolute_import', 'format_tags']


In [71]:
import sagemaker.experiments as experiments_module
print(dir(experiments_module))

['Experiment', 'Run', 'SortByType', 'SortOrderType', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_api_types', '_environment', '_helper', '_metrics', '_run_context', '_utils', 'absolute_import', 'experiment', 'list_runs', 'load_run', 'run', 'trial', 'trial_component']


In [59]:
!pip install sagemaker.experiments

Using cached sagemaker_experiments-0.1.45-py3-none-any.whl (42 kB)


In [2]:
!aws s3 ls s3://fr-detector/balanced_train/


2025-03-07 05:08:26  185655083 X_train.csv
2025-03-12 11:23:46  190159900 processed_train.csv
2025-03-07 11:15:54  185400270 train.csv
2025-03-12 13:52:52  191675855 train_final.csv
2025-03-07 05:08:29     682358 y_train.csv


In [4]:
from sagemaker.experiments.experiment import Experiment
from sagemaker.session import Session
 
experiment_name = "fraud-detection-experiment"
sagemaker_session = Session()
 
# Create experiment (if it doesn't exist)
try:
    experiment = Experiment.load(experiment_name)
    print(f"Loaded existing experiment: {experiment_name}")
except Exception as e:
    if "ResourceNotFound" in str(e):  # Experiment doesn't exist, create it
        experiment = Experiment.create(
            experiment_name=experiment_name,
            description="Credit Card Fraud Detection Experiment",
        )
        print(f"Created new experiment: {experiment_name}")
    else:
        raise  # Reraise any other error

[03/18/25 10:46:34] INFO     Found credentials from IAM Role:                                   ]8;id=727150;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=574139;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/18/25 10:46:35] INFO     Found credentials from IAM Role:                                   ]8;id=136258;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=616913;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Loaded existing experiment: fraud-detection-experiment


In [3]:
from sagemaker.experiments.run import Run
from sagemaker.session import Session
 
sagemaker_session = Session()
experiment_name = "fraud-detection-experiment"
 
with Run(experiment_name=experiment_name, sagemaker_session=sagemaker_session) as run:
    run.log_parameter("model", "xgboost")
    run.log_parameter("dataset", "train_final.csv")
    run.log_parameter("learning_rate", 0.05)
    print(f"Tracking experiment run: {run.run_name}")
 
 

[03/18/25 10:51:55] INFO     Found credentials from IAM Role:                                   ]8;id=788121;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=355963;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Tracking experiment run: sagemaker-run-1742295115-a016


In [30]:
import sagemaker
import boto3
import pandas as pd
import lightgbm as lgb
from sagemaker.session import Session
from sagemaker.s3 import S3Uploader

In [31]:
import pandas as pd  
 
# Define the S3 URI  
s3_uri = "s3://fr-detector/balanced_train/train_final_1.csv"  
 
# Load the dataset  
df = pd.read_csv(s3_uri, storage_options={"anon": False})  
 
# Display the first few rows  
df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


[03/19/25 09:38:06] INFO     Found credentials from IAM Role:                                   ]8;id=93651;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=169056;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,record_id
0,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145,-0.353703,0,0
1,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792,-0.200026,0,1
2,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288,-0.345144,0,2
3,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931,0.106129,0,3
4,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672,-0.286215,0,4


In [32]:
df.isnull().sum()

Time         0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
V8           0
V9           0
V10          0
V11          0
V12          0
V13          0
V14          0
V15          0
V16          0
V17          0
V18          0
V19          0
V20          0
V21          0
V22          0
V23          0
V24          0
V25          0
V26          0
V27          0
V28          0
Amount       0
Class        0
record_id    0
dtype: int64

In [33]:
# Define features (X) and target variable (y)
target_col = "Class"
X = df.drop(columns=[target_col])  # All columns except the target
y = df[target_col]
 
# Print shape to confirm
print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")

Feature matrix shape: (341176, 31)
Target variable shape: (341176,)


In [34]:
from sklearn.model_selection import train_test_split
 
# Split the data (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
 
# Print shapes to confirm
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")

Training set shape: (272940, 31), (272940,)
Validation set shape: (68236, 31), (68236,)


In [35]:
from sagemaker.experiments.run import Run
from sagemaker.session import Session
import lightgbm as lgb
 
# Initialize SageMaker session
sagemaker_session = Session()
experiment_name = "fraud-detection-experiment"
 
# Start tracking this training run
with Run(experiment_name=experiment_name, sagemaker_session=sagemaker_session) as run:
    # Convert dataset to LightGBM format
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
 
    # Define LightGBM parameters
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "learning_rate": 0.05,
        "num_leaves": 31,
        "max_depth": -1,
        "verbose": -1
    }
 
    # Train the model (fixed early stopping)
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],  # Validation set for monitoring
        num_boost_round=100
    )
 
    # Log training parameters and metrics in SageMaker Experiment
    run.log_parameter("learning_rate", params["learning_rate"])
    run.log_parameter("num_leaves", params["num_leaves"])
    run.log_parameter("max_depth", params["max_depth"])
 
    # Log final AUC score
    if "auc" in model.best_score["valid_0"]:
        run.log_metric("final_auc", model.best_score["valid_0"]["auc"])
 
    run.log_metric("best_iteration", model.best_iteration)
 
print("✅ Model trained successfully with experiment tracking!")

[03/19/25 09:38:28] INFO     Found credentials from IAM Role:                                   ]8;id=458339;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=620458;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

✅ Model trained successfully with experiment tracking!


In [38]:
import os
import tarfile
import boto3
 
# Define model directory
model_dir = "lightgbm_model"
os.makedirs(model_dir, exist_ok=True)
 
# Save the trained model
model_path = os.path.join(model_dir, "model.txt")
model.save_model(model_path)
 
print("✅ Model saved as model.txt")
 

✅ Model saved as model.txt


In [39]:
model.save_model("model.bst")

In [40]:
import boto3
 
s3_bucket = "fr-detector"  # Your S3 bucket name
s3_model_path = "models/model.bst"
 
s3 = boto3.client("s3")
s3.upload_file("model.bst", s3_bucket, s3_model_path)
 
print(f"Model uploaded to s3://{s3_bucket}/{s3_model_path}")

[03/19/25 09:51:28] INFO     Found credentials from IAM Role:                                   ]8;id=133991;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=958158;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Model uploaded to s3://fr-detector/models/model.bst


In [6]:
import boto3
 
sagemaker = boto3.client("sagemaker", region_name="us-east-1")
 
# Replace with your ECR Image URI and IAM Role
ecr_image_uri = "361769587425.dkr.ecr.us-east-1.amazonaws.com/lightgbm-fraud-detection:latest"
role_arn = "arn:aws:iam::361769587425:role/service-role/AmazonSageMaker-ExecutionRole-20250204T104128"
 
model_name = "lightgbm-fraud-model-v5"
 
response = sagemaker.create_model(
    ModelName=model_name,
    PrimaryContainer={
        "Image": ecr_image_uri,
        "ModelDataUrl":"s3://fr-detector/models/lightgbm_sagemaker_v2.tar.gz"
    },
    ExecutionRoleArn=role_arn
)
 
print(f"Model {model_name} created successfully!")

Model lightgbm-fraud-model-v5 created successfully!


In [7]:
endpoint_config_name = "lightgbm-fraud-endpoint-config-v5"
 
sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InstanceType": "ml.m5.large",  # Adjust as per your requirement
            "InitialInstanceCount": 1
        }
    ]
)
 
print(f"Endpoint Configuration {endpoint_config_name} created!")

Endpoint Configuration lightgbm-fraud-endpoint-config-v5 created!


In [8]:
endpoint_name = "lightgbm-fraud-endpointv5"
 
sagemaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
 
print(f"SageMaker Endpoint {endpoint_name} is being created!")

SageMaker Endpoint lightgbm-fraud-endpointv5 is being created!


In [9]:
response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
print(response["EndpointStatus"])

Creating


In [10]:
import os
model_path="/opt/ml/model/model.tar.gz"
print("m exits", os.path.exists(model_path))

m exits False


In [13]:
import os
 
print("SM_MODEL_DIR:", os.getenv("SM_MODEL_DIR"))
print("SM_MODULE_DIR:", os.getenv("SM_MODULE_DIR"))

SM_MODEL_DIR: None
SM_MODULE_DIR: None


In [14]:
import boto3
 
s3_client = boto3.client("s3")
bucket_name = "fr-detector"  # Your S3 bucket name
model_key = "models/model.tar.gz"  # Path to your model file
 
# Check if the model file exists
try:
    s3_client.head_object(Bucket=bucket_name, Key=model_key)
    print(f"✅ Model file found: s3://{bucket_name}/{model_key}")
except Exception as e:
    print(f"❌ Model file not found! Error: {e}")

✅ Model file found: s3://fr-detector/models/model.tar.gz


In [16]:
import sagemaker
 
# Get the SageMaker execution role
role = sagemaker.get_execution_role()
 
# Define your model S3 path
model_uri = "s3://fr-detector/models/model.tar.gz"
 
# Define the SageMaker container image for deployment
image_uri = "361769587425.dkr.ecr.us-east-1.amazonaws.com/lightgbm-fraud-detection:latest"  # Replace with your actual ECR image URI
 
print(f"✅ SageMaker Role: {role}")
print(f"✅ Model URI: {model_uri}")
print(f"✅ Image URI: {image_uri}")

✅ SageMaker Role: arn:aws:iam::361769587425:role/service-role/AmazonSageMaker-ExecutionRole-20250204T104128
✅ Model URI: s3://fr-detector/models/model.tar.gz
✅ Image URI: 361769587425.dkr.ecr.us-east-1.amazonaws.com/lightgbm-fraud-detection:latest


In [19]:
import boto3
import sagemaker
from sagemaker.model import Model
 
# Define variables
model_name = "lightgbm-fraud-detection-model-v7"  # Change version to avoid conflicts
execution_role = role  # Already retrieved in the previous step
image_uri = "361769587425.dkr.ecr.us-east-1.amazonaws.com/lightgbm-fraud-detection:latest"  # Replace with your actual ECR Image URI
model_uri = "s3://fr-detector/models/model.tar.gz"  # Your model path in S3
 
# Create SageMaker Model
model = Model(
    name=model_name,
    image_uri=image_uri,
    model_data=model_uri,
    role=execution_role
)
 
# Deploy the model
print(f"✅ SageMaker Model {model_name} created successfully!")

✅ SageMaker Model lightgbm-fraud-detection-model-v7 created successfully!


In [18]:
import boto3
 
sagemaker_client = boto3.client("sagemaker")
 
# List all models
response = sagemaker_client.list_models()
model_names = [model["ModelName"] for model in response["Models"]]
 
print("Available models in SageMaker:", model_names)

Available models in SageMaker: ['lightgbm-fraud-model-v5', 'lightgbm-fraud-model-v4', 'lightgbm-fraud-model-v3', 'sagemaker-scikit-learn-2025-03-19-06-35-15-311', 'lightgbm-fraud-detection-model-v2', 'lightgbm-fraud-detection-model', 'sagemaker-scikit-learn-2025-03-13-09-49-35-714', 'sagemaker-scikit-learn-2025-03-13-08-59-43-592', 'sagemaker-scikit-learn-2025-03-13-08-55-47-960', 'sagemaker-scikit-learn-2025-03-13-08-35-42-702']


In [20]:
import boto3
 
sagemaker = boto3.client("sagemaker", region_name="us-east-1")
 
# Replace with your ECR Image URI and IAM Role
ecr_image_uri = "361769587425.dkr.ecr.us-east-1.amazonaws.com/lightgbm-fraud-detection:latest"
role_arn = "arn:aws:iam::361769587425:role/service-role/AmazonSageMaker-ExecutionRole-20250204T104128"
 
model_name = "lightgbm-fraud-model-v6"
 
response = sagemaker.create_model(
    ModelName=model_name,
    PrimaryContainer={
        "Image": ecr_image_uri,
        "ModelDataUrl":"s3://fr-detector/models/lightgbm_sagemaker_v2.tar.gz"
    },
    ExecutionRoleArn=role_arn
)
 
print(f"Model {model_name} created successfully!")

Model lightgbm-fraud-model-v6 created successfully!


In [24]:
!python inference.py

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/data/neha_exploration/inference.py", line 45, in <module>
    model = load_model()
  File "/home/ec2-user/SageMaker/data/neha_exploration/inference.py", line 32, in load_model
    extract_model()  # Extract model if not already extracted
  File "/home/ec2-user/SageMaker/data/neha_exploration/inference.py", line 23, in extract_model
    raise FileNotFoundError(f"Model archive not found at {model_tar_path}")
FileNotFoundError: Model archive not found at /opt/ml/model/model.tar.gz


In [23]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.5 MB/s eta 0:00:00
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Installing backend dependencies ... one
  Preparing metadata (pyproject.toml) ... done
  Building whdone
  Created wheel for lightgbm: filename=lightgbm-4.6.0-py3-none-linux_x86_64.whl size=2737777 sha256=c74086ca353c068b739be0f4dc0b08e71bda9863a7630dc5a3208b2de720b111
  Stored in directory: /home/ec2-user/.cache/pip/wheels/bb/db/6d/7814aed03437129dc284a055c084f201b765deb54b6908efab
Successfully built lightgbm


In [43]:
from sagemaker.model import Model
 
model_data = f"s3://{s3_bucket}/{s3_model_path}"
 
# Define a SageMaker model
lgb_model = Model(
image_uri="683313688378.dkr.ecr.us-east-1.amazonaws.com/lightgbm:latest",
    model_data=model_data,
    role= sagemaker.get_execution_role(),
)
 
# Deploy endpoint
predictor = lgb_model.deploy(initial_instance_count=1, instance_type="ml.m5.large")
 
print("Model deployed successfully!")

[03/19/25 10:01:46] INFO     Creating model with name: lightgbm-2025-03-19-10-01-46-767             ]8;id=793869;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=549463;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 )                                                                                           │
│   11                                                                                             │
│   12 # Deploy endpoint                                                                           │
│ ❱ 13 predictor = lgb_model.deploy(initial_instance_count=1, instance_type="ml.m5.large")         │
│   14                                                                                             │
│   15 print("Model deployed successfully!")                                                       │
│   16                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:1728 in    │
│ deploy                                                                                           │
│                                                                                                  │
│   1725 │   │   │   return None                                                                   │
│   1726 │   │                                                                                     │
│   1727 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1728 │   │   │   self._create_sagemaker_model(                                                 │
│   1729 │   │   │   │   instance_type=instance_type,                                              │
│   1730 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1731 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:985 in     │
│ _create_sagemaker_model                                                                          │
│                                                                                                  │
│    982 │   │   │   │   enable_network_isolation=self._enable_network_isolation,                  │
│    983 │   │   │   │   tags=format_tags(tags),                                                   │
│    984 │   │   │   )                                                                             │
│ ❱  985 │   │   │   self.sagemaker_session.create_model(**create_model_args)                      │
│    986 │                                                                                         │
│    987 │   def _get_model_uri(self):                                                             │
│    988 │   │   model_uri = None                                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:4109 in  │
│ create_model                                                                                     │
│                                                                                                  │
│   4106 │   │   │   │   else:                                                                     │
│   4107 │   │   │   │   │   raise                                                                 │
│   4108 │   │                                                                                     │
│ ❱ 4109 │   │   self._intercept_create_request(create_model_

In [47]:
import boto3
import tarfile
import os
from sagemaker.model import Model
 
# Define S3 URIs
s3_model_bst_uri = "s3://fr-detector/models/model.bst"  # Model file in S3
s3_model_tar_uri = "s3://fr-detector/models/model.tar.gz"  # Path for the tar file
 
# Extract bucket name and key from S3 URI
s3_bucket = s3_model_bst_uri.split('/')[2]
s3_bst_key = "/".join(s3_model_bst_uri.split('/')[3:])  # Path inside S3
 
s3_tar_key = "/".join(s3_model_tar_uri.split('/')[3:])  # Path for tar.gz in S3
 
# Initialize S3 client
s3 = boto3.client("s3")
 
# Step 1: Download model.bst from S3
print("Downloading model.bst from S3...")
s3.download_file(s3_bucket, s3_bst_key, "model.bst")
 
# Step 2: Package model into model.tar.gz
print("Packaging model into model.tar.gz...")
os.makedirs("model", exist_ok=True)
os.rename("model.bst", "model/model.bst")
 
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("model", arcname=".")
 
# Step 3: Upload model.tar.gz to the same S3 bucket
print("Uploading model.tar.gz to S3...")
s3.upload_file("model.tar.gz", s3_bucket, s3_tar_key)
 
# Step 4: Deploy the model in SageMaker
role_arn = "arn:aws:iam::361769587425:role/service-role/AmazonSageMaker-ExecutionRole-20250204T104128"  # Replace with actual SageMaker role
image_uri = "361769587425.dkr.ecr.us-east-1.amazonaws.com/lightgbm-fraud-detection"
 
print("Deploying the model on SageMaker...")
model = Model(
    image_uri=image_uri,
    model_data=s3_model_tar_uri,  # Direct S3 URI for tar.gz
    role=role_arn,
)
 
predictor = model.deploy(instance_type="ml.m5.large", initial_instance_count=1)
 
# Step 5: Print endpoint details
print("Model deployed successfully!")
print(f"Endpoint name: {predictor.endpoint_name}")

Packaging model into model.tar.gz...
Uploading model.tar.gz to S3...
Deploying the model on SageMaker...


[03/19/25 10:21:47] INFO     Creating model with name:                                              ]8;id=478784;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=106864;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             lightgbm-fraud-detection-2025-03-19-10-21-47-337                                      

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:46                                                                                   │
│                                                                                                  │
│   43 │   role=role_arn,                                                                          │
│   44 )                                                                                           │
│   45                                                                                             │
│ ❱ 46 predictor = model.deploy(instance_type="ml.m5.large", initial_instance_count=1)             │
│   47                                                                                             │
│   48 # Step 5: Print endpoint details                                                            │
│   49 print("Model deployed successfully!")                                                       │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:1728 in    │
│ deploy                                                                                           │
│                                                                                                  │
│   1725 │   │   │   return None                                                                   │
│   1726 │   │                                                                                     │
│   1727 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1728 │   │   │   self._create_sagemaker_model(                                                 │
│   1729 │   │   │   │   instance_type=instance_type,                                              │
│   1730 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1731 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:985 in     │
│ _create_sagemaker_model                                                                          │
│                                                                                                  │
│    982 │   │   │   │   enable_network_isolation=self._enable_network_isolation,                  │
│    983 │   │   │   │   tags=format_tags(tags),                                                   │
│    984 │   │   │   )                                                                             │
│ ❱  985 │   │   │   self.sagemaker_session.create_model(**create_model_args)                      │
│    986 │                                                                                         │
│    987 │   def _get_model_uri(self):                                                             │
│    988 │   │   model_uri = None                                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:4109 in  │
│ create_model                                                                                     │
│                                                                                                  │
│   4106 │   │   │   │   else:                                                                     │
│   4107 │   │   │   │   │   raise                                                                 │
│   4108 │   │                                                                                     │
│ ❱ 4109 │   │   self._intercept_create_request(create_model_

In [52]:
import boto3
 
# Define S3 bucket and model path
s3_bucket = "fr-detector"  # Your S3 bucket name
s3_model_path = f"models/lightgbm_model.tar.gz"  # Model path inside the bucket
 
# Upload to S3
s3_client = boto3.client("s3")
s3_client.upload_file("lightgbm_model.tar.gz", s3_bucket, s3_model_path)
 
print(f"✅ Model uploaded successfully to s3://{s3_bucket}/{s3_model_path}")

✅ Model uploaded successfully to s3://fr-detector/models/lightgbm_model.tar.gz


In [55]:
import boto3
 
s3_client = boto3.client("s3")
s3_bucket = "fr-detector"
s3_model_key = "models/lightgbm_model.tar.gz"
 
# Check if model exists in S3
try:
    s3_client.head_object(Bucket=s3_bucket, Key=s3_model_key)
    print(f"✅ Model exists at s3://{s3_bucket}/{s3_model_key}")
except Exception as e:
    print(f"❌ Model not found in S3: {e}")

✅ Model exists at s3://fr-detector/models/lightgbm_model.tar.gz


In [56]:
%%writefile inference.py
 
import os
import lightgbm as lgb
import json
import numpy as np
import pandas as pd
import joblib
 
# Model loading function
def model_fn(model_dir):
    """Loads the trained LightGBM model from disk."""
    model_path = os.path.join(model_dir, "model.txt")
    model = lgb.Booster(model_file=model_path)
    return model
 
# Input processing function
def input_fn(request_body, request_content_type):
    """Converts input JSON into a Pandas DataFrame."""
    if request_content_type == "application/json":
        data = json.loads(request_body)
        return pd.DataFrame(data)
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")
 
# Prediction function
def predict_fn(input_data, model):
    """Runs inference and returns predictions."""
    predictions = model.predict(input_data)
    return predictions.tolist()
 
# Output formatting function
def output_fn(predictions, response_content_type):
    """Formats the output as JSON."""
    if response_content_type == "application/json":
        return json.dumps(predictions)
    else:
        raise ValueError(f"Unsupported response type: {response_content_type}")

Writing inference.py


In [59]:
import os
import shutil
 
# Define model directory
model_dir = "lightgbm_sagemaker"
os.makedirs(model_dir, exist_ok=True)
 
# Move model.txt and inference.py to this directory
shutil.move("lightgbm_model/model.txt", os.path.join(model_dir, "model.txt"))
shutil.move("inference.py", os.path.join(model_dir, "inference.py"))
 
print("✅ Model directory prepared!")

✅ Model directory prepared!


In [60]:
import tarfile
 
tar_path = "lightgbm_sagemaker.tar.gz"
with tarfile.open(tar_path, "w:gz") as tar:
    tar.add(model_dir, arcname=".")
 
print("✅ Model and inference script archived successfully!")
 

✅ Model and inference script archived successfully!


In [61]:
import boto3
 
# Define S3 bucket and model path
s3_bucket = "fr-detector"  # Your S3 bucket name
s3_model_path = f"models/lightgbm_sagemaker.tar.gz"  # Path inside the bucket
 
# Upload to S3
s3_client = boto3.client("s3")
s3_client.upload_file("lightgbm_sagemaker.tar.gz", s3_bucket, s3_model_path)
 
print(f"✅ Model uploaded successfully to s3://{s3_bucket}/{s3_model_path}")

✅ Model uploaded successfully to s3://fr-detector/models/lightgbm_sagemaker.tar.gz


In [62]:
import sagemaker
from sagemaker.model import Model
 
# Define model S3 path
s3_model_uri = f"s3://fr-detector/models/lightgbm_sagemaker.tar.gz"
 
# Get SageMaker execution role
role = sagemaker.get_execution_role()
 
# Define the container for deployment (using a generic Python container)
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3"
 
# Create a SageMaker Model
lightgbm_model = Model(
    model_data=s3_model_uri,
    image_uri=image_uri,
    role=role,
    predictor_cls=sagemaker.predictor.Predictor
)
 
print("✅ SageMaker Model created successfully!")

✅ SageMaker Model created successfully!


In [63]:
print("hello")

hello


In [64]:
import boto3
 
sm_client = boto3.client("sagemaker")
 
# List all models
response = sm_client.list_models()
 
# Print model names and creation time
for model in response["Models"]:
    print(f"Model Name: {model['ModelName']}, Created At: {model['CreationTime']}")

Model Name: sagemaker-scikit-learn-2025-03-13-10-45-28-137, Created At: 2025-03-13 10:45:28.626000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-10-45-05-048, Created At: 2025-03-13 10:45:05.531000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-10-40-51-005, Created At: 2025-03-13 10:40:51.453000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-09-49-35-714, Created At: 2025-03-13 09:49:36.243000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-59-43-592, Created At: 2025-03-13 08:59:44.115000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-55-47-960, Created At: 2025-03-13 08:55:48.453000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-35-42-702, Created At: 2025-03-13 08:35:43.195000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-35-13-607, Created At: 2025-03-13 08:35:14.065000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-20-54-313, Created At: 2025-03-13 08:20:54.845000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-11-10-

In [65]:
import boto3
 
sm_client = boto3.client("sagemaker")
 
# Get the most recent model
response = sm_client.list_models(SortBy="CreationTime", SortOrder="Descending", MaxResults=5)
 
# Print model names and creation time
for model in response["Models"]:
    print(f"Model Name: {model['ModelName']}, Created At: {model['CreationTime']}")

Model Name: sagemaker-scikit-learn-2025-03-13-10-45-28-137, Created At: 2025-03-13 10:45:28.626000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-10-45-05-048, Created At: 2025-03-13 10:45:05.531000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-10-40-51-005, Created At: 2025-03-13 10:40:51.453000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-09-49-35-714, Created At: 2025-03-13 09:49:36.243000+00:00
Model Name: sagemaker-scikit-learn-2025-03-13-08-59-43-592, Created At: 2025-03-13 08:59:44.115000+00:00


In [66]:
import sagemaker
from sagemaker.model import Model
 
# Define model name explicitly
model_name = "lightgbm-fraud-detection-model"
 
# Define S3 path for the model
s3_model_uri = "s3://fr-detector/models/lightgbm_sagemaker.tar.gz"
 
# Get SageMaker execution role
role = sagemaker.get_execution_role()
 
# Define the container for deployment (using a generic Python container)
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3"
 
# Create the SageMaker model
lightgbm_model = Model(
    model_data=s3_model_uri,
    image_uri=image_uri,
    role=role,
    predictor_cls=sagemaker.predictor.Predictor,
    name=model_name  # Explicitly setting the model name
)
 
# Register the model in SageMaker
lightgbm_model.create()
print(f"✅ SageMaker Model '{model_name}' created successfully!")

[03/18/25 14:48:50] INFO     Creating model with name: lightgbm-fraud-detection-model               ]8;id=688669;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=2678;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

✅ SageMaker Model 'lightgbm-fraud-detection-model' created successfully!


In [67]:
import boto3
 
sm_client = boto3.client("sagemaker")
 
# Define endpoint configuration name
endpoint_config_name = "lightgbm-fraud-detection-endpoint-config"
 
# Create endpoint configuration
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": "lightgbm-fraud-detection-model",  # The model name we registered
            "InstanceType": "ml.m5.large",  # Change instance type if needed
            "InitialInstanceCount": 1
        }
    ]
)
 
print(f"✅ Endpoint configuration '{endpoint_config_name}' created successfully!")

✅ Endpoint configuration 'lightgbm-fraud-detection-endpoint-config' created successfully!


In [68]:
# Define endpoint name
endpoint_name = "lightgbm-fraud-detection-endpoint"
 
# Create the endpoint
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName="lightgbm-fraud-detection-endpoint-config"  # Use the config we created
)
 
print(f"🚀 SageMaker Endpoint '{endpoint_name}' is being deployed! This may take a few minutes...")

🚀 SageMaker Endpoint 'lightgbm-fraud-detection-endpoint' is being deployed! This may take a few minutes...


In [69]:
import time
 
# Function to check endpoint status
def check_endpoint_status(endpoint_name):
    while True:
        response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = response["EndpointStatus"]
        print(f"🚀 Endpoint Status: {status}")
 
        if status in ["InService", "Failed"]:
            break  # Stop when deployment is complete or failed
        time.sleep(30)  # Wait 30 seconds before checking again
 
# Check status of the deployed endpoint
check_endpoint_status("lightgbm-fraud-detection-endpoint")

🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: Creating
🚀 Endpoint Status: C

In [70]:
response = sm_client.describe_endpoint(EndpointName="lightgbm-fraud-detection-endpoint")
print("🚨 Failure Reason:", response["FailureReason"])

🚨 Failure Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.


In [37]:
from sagemaker.analytics import ExperimentAnalytics
 
experiment_name = "fraud-detection-experiment"
 
analytics = ExperimentAnalytics(experiment_name=experiment_name)
analytics.dataframe()

[03/19/25 09:46:47] INFO     Found credentials from IAM Role:                                   ]8;id=539349;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=579729;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

,TrialComponentName,DisplayName,learning_rate,max_depth,num_leaves,final_auc - Min,final_auc - Max,final_auc - Avg,final_auc - StdDev,final_auc - Last,...,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,eta,gamma,min_child_weight,silent,subsample
0,fraud-detection-experiment-sagemaker-run-17423...,fraud-detection-experiment-sagemaker-run-17423...,0.05,-1.0,31.0,0.999889,0.999889,0.999889,0.0,0.999889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fraud-detection-experiment-sagemaker-run-17423...,fraud-detection-experiment-sagemaker-run-17423...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fraud-detection-experiment-sagemaker-run-17423...,fraud-detection-experiment-sagemaker-run-17423...,0.05,-1.0,31.0,0.999889,0.999889,0.999889,0.0,0.999889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,fraud-detection-experiment-sagemaker-run-17423...,fraud-detection-experiment-sagemaker-run-17423...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,fraud-detection-experiment-sagemaker-run-17423...,fraud-detection-experiment-sagemaker-run-17423...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,fraud-detection-experiment-sagemaker-run-17422...,fraud-detection-experiment-sagemaker-run-17422...,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,sagemaker-xgboost-2025-03-12-13-40-24-161-aws-...,sagemaker-xgboost-2025-03-12-13-40-24-161-aws-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,s3://fr-detector/balanced_train/train_final.csv,NaN,s3://fr-detector/models/,NaN,s3://fr-detector/models/sagemaker-xgboost-2025...,NaN,NaN,NaN,NaN,NaN
7,fraud-detection-experiment-trial-xgboost-17417...,fraud-detection-experiment-trial-xgboost-17417...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,fraud-detection-experiment-fraud-detection-run...,fraud-detection-experiment-fraud-detection-run...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,xgboost-2025-03-12-12-41-47-901-aws-training-job,xgboost-2025-03-12-12-41-47-901-aws-training-job,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,s3://fr-detector/balanced_train/processed_trai...,NaN,s3://fr-detector/xgboost_model,NaN,s3://fr-detector/xgboost_model/xgboost-2025-03...,0.2,4.0,6.0,0.0,0.7


In [ ]:
#DEBUGGING

In [75]:
import os
import lightgbm as lgb
 
def model_fn(model_dir):
    """Loads the trained LightGBM model from disk."""
    model_path = os.path.join(model_dir, "model.txt")
    
    # Debugging statement
    print(f"Loading model from: {model_path}")
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")
    
    model = lgb.Booster(model_file=model_path)
    print("Model loaded successfully!")
    return model

In [82]:
!python -m py_compile lightgbm_sagemaker/inference.py

In [77]:
model=model_fn("lightgbm_sagemaker")

Loading model from: lightgbm_sagemaker/model.txt
Model loaded successfully!


In [79]:
import json
import pandas as pd
 
def input_fn(request_body, request_content_type):
    """Converts input JSON into a Pandas DataFrame."""
    
    print(f"Received request with content type: {request_content_type}")
    
    if request_content_type == "application/json":
        data = json.loads(request_body)
        
        print(f"Parsed JSON: {data}")
        
        df = pd.DataFrame(data)
        print("Converted to DataFrame:")
        print(df.head())  # Show first few rows
        
        return df
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")
 

In [80]:
test_input = json.dumps({
    "V14": [0.5],
    "V4": [1.2],
    "V11": [-0.8],
    "V9": [0.3],
    "V7": [-1.5],
    "V2": [0.7],
    "V18": [0.9],
    "V1": [-0.2],
    "V6": [1.1],
    "V5": [-0.4],
    "V19": [0.8],
    "EV90": [0.6]
})  
 
df = input_fn(test_input, "application/json")

Received request with content type: application/json
Parsed JSON: {'V14': [0.5], 'V4': [1.2], 'V11': [-0.8], 'V9': [0.3], 'V7': [-1.5], 'V2': [0.7], 'V18': [0.9], 'V1': [-0.2], 'V6': [1.1], 'V5': [-0.4], 'V19': [0.8], 'EV90': [0.6]}
Converted to DataFrame:
   V14   V4  V11   V9   V7   V2  V18   V1   V6   V5  V19  EV90
0  0.5  1.2 -0.8  0.3 -1.5  0.7  0.9 -0.2  1.1 -0.4  0.8   0.6


In [83]:
import os
import lightgbm as lgb
import json
import numpy as np
import pandas as pd
import joblib
 
# Model loading function
# def model_fn(model_dir):
#     """Loads the trained LightGBM model from disk."""
#     model_path = os.path.join(model_dir, "model.txt")
#     model = lgb.Booster(model_file=model_path)
#     return model

def model_fn(model_dir):
    """Loads the trained LightGBM model from disk."""
    model_path = os.path.join(model_dir, "model.txt")
    
    # Debugging statement
    print(f"Loading model from: {model_path}")
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")
    
    model = lgb.Booster(model_file=model_path)
    print("Model loaded successfully!")
    return model
 
# Input processing function
# def input_fn(request_body, request_content_type):
#     """Converts input JSON into a Pandas DataFrame."""
#     if request_content_type == "application/json":
#         data = json.loads(request_body)
#         return pd.DataFrame(data)
#     else:
#         raise ValueError(f"Unsupported content type: {request_content_type}")

def input_fn(request_body, request_content_type):
    """Converts input JSON into a Pandas DataFrame."""
    
    print(f"Received request with content type: {request_content_type}")
    
    if request_content_type == "application/json":
        data = json.loads(request_body)
        
        print(f"Parsed JSON: {data}")
        
        df = pd.DataFrame(data)
        print("Converted to DataFrame:")
        print(df.head())  # Show first few rows
        
        return df
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")
 
# Prediction function
def predict_fn(input_data, model):
    """Runs inference and returns predictions."""
    predictions = model.predict(input_data)
    return predictions.tolist()
 
# Output formatting function
def output_fn(predictions, response_content_type):
    """Formats the output as JSON."""
    if response_content_type == "application/json":
        return json.dumps(predictions)
    else:
        raise ValueError(f"Unsupported response type: {response_content_type}")

In [88]:
predictions=predict_fn(df,model)
print("predictions:", predictions)

predictions: [0.0023384551914107437]


In [87]:
print(model.feature_name())

['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'record_id']


In [86]:
import json
 
test_input = json.dumps({
    "Time": [0.5], "V1": [1.2], "V2": [-0.8], "V3": [0.3], "V4": [1.5], "V5": [0.7], "V6": [0.9], "V7": [1.1], "V8": [-0.2], "V9": [1.1],
    "V10": [-0.4], "V11": [0.8], "V12": [0.6], "V13": [-0.1], "V14": [1.3], "V15": [0.2], "V16": [0.9], "V17": [-0.5], "V18": [0.7], "V19": [-1.2],
    "V20": [0.4], "V21": [0.6], "V22": [-0.3], "V23": [0.8], "V24": [1.0], "V25": [-0.9], "V26": [1.2], "V27": [0.5], "V28": [-1.3], "Amount": [50.0],
    "record_id": [12345]
})
 
df = input_fn(test_input, "application/json")

Received request with content type: application/json
Parsed JSON: {'Time': [0.5], 'V1': [1.2], 'V2': [-0.8], 'V3': [0.3], 'V4': [1.5], 'V5': [0.7], 'V6': [0.9], 'V7': [1.1], 'V8': [-0.2], 'V9': [1.1], 'V10': [-0.4], 'V11': [0.8], 'V12': [0.6], 'V13': [-0.1], 'V14': [1.3], 'V15': [0.2], 'V16': [0.9], 'V17': [-0.5], 'V18': [0.7], 'V19': [-1.2], 'V20': [0.4], 'V21': [0.6], 'V22': [-0.3], 'V23': [0.8], 'V24': [1.0], 'V25': [-0.9], 'V26': [1.2], 'V27': [0.5], 'V28': [-1.3], 'Amount': [50.0], 'record_id': [12345]}
Converted to DataFrame:
   Time   V1   V2   V3   V4   V5   V6   V7   V8   V9  ...  V21  V22  V23  V24  \
0   0.5  1.2 -0.8  0.3  1.5  0.7  0.9  1.1 -0.2  1.1  ...  0.6 -0.3  0.8  1.0   

   V25  V26  V27  V28  Amount  record_id  
0 -0.9  1.2  0.5 -1.3    50.0      12345  

[1 rows x 31 columns]


In [ ]:
#ANOTHER ATTEMPT

In [104]:
import boto3
 
# Define S3 bucket and model path
s3_bucket = "fr-detector"  # Your S3 bucket name
s3_model_path = "models/lightgbm_sagemaker_v2.tar.gz"  # S3 path
local_model_file = "lightgbm_sagemaker/model_v2.tar.gz"  # Local file
 
# Initialize S3 client
s3_client = boto3.client("s3")
 
# Upload model to S3
s3_client.upload_file(local_model_file, s3_bucket, s3_model_path)
 
print(f"✅ Model uploaded successfully to: s3://{s3_bucket}/{s3_model_path}")

✅ Model uploaded successfully to: s3://fr-detector/models/lightgbm_sagemaker_v2.tar.gz


In [98]:
import tarfile
import os
 
# Define file names
model_dir = "lightgbm_sagemaker"  # Update this path if needed
model_file = os.path.join(model_dir, "model.txt")
inference_file = os.path.join(model_dir, "inference.py")
archive_file = os.path.join(model_dir, "model_v2.tar.gz")  # New archive name
 
# Ensure both files exist before archiving
if not os.path.exists(model_file):
    raise FileNotFoundError(f"Model file not found at {model_file}")
 
if not os.path.exists(inference_file):
    raise FileNotFoundError(f"Inference script not found at {inference_file}")
 
# Create tar.gz archive
with tarfile.open(archive_file, "w:gz") as tar:
    tar.add(model_file, arcname="model.txt")
    tar.add(inference_file, arcname="inference.py")
 
print(f"✅ Archive created: {archive_file}")
 

✅ Archive created: lightgbm_sagemaker/model_v2.tar.gz


In [105]:
import sagemaker
from sagemaker.model import Model
from sagemaker.predictor import Predictor
 
# Define S3 model path (Use your updated model path)
s3_model_uri = "s3://fr-detector/models/lightgbm_sagemaker_v2.tar.gz"
 
# Get SageMaker execution role
role = sagemaker.get_execution_role()
 
# Define the SageMaker container for deployment
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3"
 
# Define Model and Endpoint Names
model_name = "lightgbm-fraud-detection-model-v2"
endpoint_name = "lightgbm-fraud-detection-endpoint-v2"
 
# Create a SageMaker Model with a specific name
lightgbm_model = Model(
    model_data=s3_model_uri,
    role=role,
    image_uri=image_uri,
    predictor_cls=Predictor,
    name=model_name  # Explicitly naming the model
)
 
print(f"✅ SageMaker Model '{model_name}' created successfully!")
 
# Deploy the model as an endpoint with a specific name
predictor = lightgbm_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name  # Explicitly naming the endpoint
)
 
print(f"✅ Model deployed successfully with endpoint: {endpoint_name}")

✅ SageMaker Model 'lightgbm-fraud-detection-model-v2' created successfully!


[03/18/25 16:41:58] INFO     Creating model with name: lightgbm-fraud-detection-model-v2            ]8;id=204859;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=343360;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/18/25 16:41:59] INFO     Creating endpoint-config with name                                     ]8;id=922820;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=870491;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5937\5937]8;;\
                             lightgbm-fraud-detection-endpoint-v2                                                  

                    INFO     Creating endpoint with name lightgbm-fraud-detection-endpoint-v2       ]8;id=177566;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=991929;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4759\4759]8;;\

--------------------------------------------------*

[03/18/25 17:07:35] ERROR    Please check the troubleshooting guide for common errors:              ]8;id=124757;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=233212;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4782\4782]8;;\
                             https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-t                
                             roubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoi                
                             nt                                                                                    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:30                                                                                   │
│                                                                                                  │
│   27 print(f"✅ SageMaker Model '{model_name}' created successfully!")                           │
│   28                                                                                             │
│   29 # Deploy the model as an endpoint with a specific name                                      │
│ ❱ 30 predictor = lightgbm_model.deploy(                                                          │
│   31 │   initial_instance_count=1,                                                               │
│   32 │   instance_type="ml.m5.large",                                                            │
│   33 │   endpoint_name=endpoint_name  # Explicitly naming the endpoint                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:1786 in    │
│ deploy                                                                                           │
│                                                                                                  │
│   1783 │   │   │   if is_explainer_enabled:                                                      │
│   1784 │   │   │   │   explainer_config_dict = explainer_config._to_request_dict()               │
│   1785 │   │   │                                                                                 │
│ ❱ 1786 │   │   │   self.sagemaker_session.endpoint_from_production_variants(                     │
│   1787 │   │   │   │   name=self.endpoint_name,                                                  │
│   1788 │   │   │   │   production_variants=[production_variant],                                 │
│   1789 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:5940 in  │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   5937 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   5938 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   5939 │   │                                                                                     │
│ ❱ 5940 │   │   return self.create_endpoint(                                                      │
│   5941 │   │   │   endpoint_name=name,                                                           │
│   5942 │   │   │   config_name=name,                                                             │
│   5943 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:4785 in  │
│ create_endpoint                                                                                  │
│                                                                                                  │
│   4782 │   │   │   logger.error(                                                                 │
│   4783 │   │   │   │   "Please check the troubleshooting guide for common errors: %s", troubles  │
│   4784 │   │   │   )                                                                             │
│ ❱ 4785 │   │   │   raise e                                  

In [1]:
import tarfile
 
# Path to the tar.gz file
tar_path = "lightgbm_sagemaker_v2.tar.gz"
 
# Extract model.txt
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extract("model.txt", path=".")
 
print("✅ Extracted 'model.txt' from archive")

✅ Extracted 'model.txt' from archive


In [4]:
import lightgbm as lgb
 
# Load model from model.txt
booster = lgb.Booster(model_file="model.txt")
 
# Save model in .bst format
booster.save_model("lightgbm_model.bst")
 
print("✅ Converted 'model.txt' to 'lightgbm_model.bst'")

✅ Converted 'model.txt' to 'lightgbm_model.bst'


In [3]:
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.1 MB/s eta 0:00:00
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Installing backend dependencies ... one
  Preparing metadata (pyproject.toml) ... done
  Building wdone
  Created wheel for lightgbm: filename=lightgbm-4.6.0-py3-none-linux_x86_64.whl size=2737778 sha256=321d55061c71c748da2c4deac0872a46e5248eee19d5a40fabad9ebe6760cf33
  Stored in directory: /home/ec2-user/.cache/pip/wheels/bb/db/6d/7814aed03437129dc284a055c084f201b765deb54b6908efab
Successfully built lightgbm


In [ ]:
import boto3
 
s3_client = boto3.client("s3")
bucket_name = "fr-detector"
s3_model_path = "models/lightgbm_model.bst"
 
# Upload the file
s3_client.upload_file("lightgbm_model.bst", bucket_name, s3_model_path)
 
print(f"✅ Model uploaded to S3: s3://{bucket_name}/{s3_model_path}")

In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker.predictor import Predictor
 
# Updated S3 model path
s3_model_uri = "s3://fr-detector/models/lightgbm_model.bst"
 
# Get SageMaker execution role
role = sagemaker.get_execution_role()
 
# Use XGBoost container for deployment
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1"
 
# Define Model and Endpoint Names
model_name = "lightgbm-fraud-detection-model-v2"
endpoint_name = "lightgbm-fraud-detection-endpoint-v2"
 
# Create Model object
lightgbm_model = Model(
    model_data=s3_model_uri,
    role=role,
    image_uri=image_uri,
    predictor_cls=Predictor,
    name=model_name
)
 
print(f"✅ SageMaker Model '{model_name}' created successfully!")
 
# Deploy model
predictor = lightgbm_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)
 
print(f"✅ Model deployed successfully with endpoint: {endpoint_name}")

In [ ]:
#docker container method

In [108]:
import boto3
import os
 
# Define S3 URI
s3_uri = "s3://fr-detector/models/lightgbm_sagemaker_v2.tar.gz"
 
# Extract bucket name and key from S3 URI
parsed_url = s3_uri.replace("s3://", "").split("/", 1)
s3_bucket = parsed_url[0]
s3_key = parsed_url[1]
 
# Define local file path
local_file_path = os.path.basename(s3_key)  # Saves with same filename
 
# Initialize S3 client
s3_client = boto3.client("s3")
 
# Download the model file from S3
s3_client.download_file(s3_bucket, s3_key, local_file_path)
 
print(f"✅ Model file downloaded successfully: {local_file_path}")

✅ Model file downloaded successfully: lightgbm_sagemaker_v2.tar.gz


In [21]:
!python train_script.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [09:08:12] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
F1 Score: 0.9993622308724242
Error during training: [Errno 13] Permission denied: '/opt/ml/model'
Traceback (most recent cal

In [22]:
!pip install xgboost

In [25]:
import argparse
 
parser = argparse.ArgumentParser()
parser.add_argument("--train-file", type=str, default="/opt/ml/input/data/train/train_final.csv")
parser.add_argument("--output-data-dir", type=str, default="/opt/ml/model")
parser.add_argument("--model-dir", type=str, default="/opt/ml/model")
parser.add_argument("--eval-metric", type=str, default="auc")
parser.add_argument("--num-round", type=int, default=100)
parser.add_argument("--objective", type=str, default="binary:logistic")
 
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--train-file TRAIN_FILE]
                             [--output-data-dir OUTPUT_DATA_DIR]
                             [--model-dir MODEL_DIR]
                             [--eval-metric EVAL_METRIC]
                             [--num-round NUM_ROUND] [--objective OBJECTIVE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/ec2-user/.local/share/jupyter/runtime/kernel-7435812a-182e-403f-a390-e52748ec5136.json


SystemExit: 2

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [26]:
import os
print("checking if train_final.csv exists",os.path.exists(args.train_file))

NameError: name 'args' is not defined

In [17]:
!python dummy.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ec2-user/SageMaker/data/neha_exploration/dummy.py:21: UserWarning: [08:50:39] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  model.save_model(model_path)
Model saved at: ./model/xgboost_model.bst


In [18]:
!ls -l ./model

total 608
-rw-rw-r-- 1 ec2-user ec2-user 469027 Mar  7 12:32 xgb_model.json
-rw-rw-r-- 1 ec2-user ec2-user  14773 Mar 19 08:50 xgboost_model.bst
-rw-rw-r-- 1 ec2-user ec2-user 132021 Mar  7 12:32 xgboost-model.tar.gz
